# Preliminaries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

sys.path.append("/Users/paolo/Documents/methods/CMI_FS")
from feature_selection import forwardFeatureSelection

sys.path.append("/Users/paolo/Documents/methods/LinCFA")
from LinCFA import LinCFA

sys.path.append("/Users/paolo/Documents/methods/NonLinCFA")
from NonLinCFA import NonLinCFA

sys.path.append("/Users/paolo/Documents/Droughts/Paolo/regression_NonLinCFA")
from aux import standardize,unfold_dataset,compute_r2,prepare_target,prepare_features,aggregate_unfolded_data,aggregate_unfolded_data_onlyTrain,FS_with_linearWrapper,compare_methods, compute_r2, aggregate_data_withoutUnfolding


# Piemonte Nord e Sud, Adda, Ticino, Dora

## Only temp+prec

In [3]:
### BOTH
#basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
basins = ['Piemonte_Nord','Piemonte_Sud','Adda','Dora','Ticino']
path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
path_features='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/temporal_aggreg/'

for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')

    eps = 0.001
    actual_path = path_features+basin+'_aggreg.csv'
        
    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg', 
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w', 
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w', 
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr', 
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w', 
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w', 
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
        
    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    
    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
    res = {
            "delta" : [], 
            "numSelected" : [], 
            "selectedFeatures" : [] 
        }
        
    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
        
    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]
    
    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    
    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
        

####################Piemonte_Nord####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.278983    0.00  2001     1 -1.146332
1    2001-01-13  0.494910    0.51  2001     2  0.371173
2    2001-01-21  0.496092    0.51  2001     3  0.379474
3    2001-01-29  0.427992    0.43  2001     5 -0.099118
4    2001-02-06  0.400512    0.41  2001     6 -0.292244
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.363952    0.37  2009    48 -0.549184
407  2009-12-05  0.400487    0.40  2009    49 -0.292423
408  2009-12-13  0.506771    0.52  2009    50  0.454529
409  2009-12-21  0.387530    0.53  2009    52 -0.383480
410  2009-12-29  0.279894    0.27  2009    53 -1.139931

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 89

Number of aggregated features: 6

Number of features: 89

Number of aggregated features: 7

Number of features: 89

Number of aggregated features: 7

Num

----- MI Scores -----
[(0, 0.10828140105633417), (42, 0.10540351107087807), (3, 0.10360024919751139), (1, 0.09732112095987412), (59, 0.09632459933985814), (51, 0.09504862903894257), (50, 0.09271731107050801), (2, 0.09221894148727858), (61, 0.09118775336609275), (55, 0.08847082224879566), (52, 0.08806435647689961), (41, 0.08736519266509175), (53, 0.08071317337658432), (47, 0.07680037628099362), (43, 0.075071626108866), (58, 0.07423403064202792), (4, 0.07235450820565101), (56, 0.07218705423176484), (60, 0.0697176298548119), (57, 0.06961542865992468), (45, 0.06924061086735848), (46, 0.06759455518696374), (39, 0.06692499909581603), (54, 0.06392844522143308), (5, 0.0600274074676265), (33, 0.05591576054731865), (36, 0.05220501936022941), (49, 0.048093215687404), (8, 0.046018442921380305), (40, 0.04583156722857606), (34, 0.044951332652584115), (30, 0.04309325411575538), (35, 0.04306263516036035), (7, 0.042308250947635566), (38, 0.03798035132289578), (32, 0.037792474675920285), (6, 0.036413777

CMI: 0.006624388253266861
CMI: 0.0030981634182589424
CMI: 0.006282864570508945
CMI: 0.0029264662653806783
CMI: 0.014122997852305863
CMI: 0.006113992740532287
CMI: 0.0030719299225818453
CMI: 0.0024262826772420976
CMI: 0.0010753876543792146
CMI: 0.02810098729551408
CMI: 0.024144132254435985
CMI: 0.012342105593410813
CMI: 0.006700675919893262
CMI: 0.003896541867466491
CMI: 0.002893936060894886
CMI: 0.0003093615834881408
CMI: 0.010107600077330414
CMI: 0.0076079727244009465
CMI: 0.01908967461476177
CMI: 0.012462386718424756
CMI: 0.009210478531231081
CMI: 0.011094173889947156
CMI: 0.014111781613090957
CMI: 0.03333163300111361
CMI: 0.01961083349465806
CMI: 0.01130387795574253
CMI: 0.0034721342846523173
CMI: 0.0351507953297725
CMI: 0.01716450610747053
CMI: 0.023456973403961967
Highest CMI score: 0.0351507953297725
Adding original feature: 45
CMI: 0.003828758131675769
CMI: 0.001600282747443449
CMI: 0.0006175627037483256
CMI: 0.014636880627741128
CMI: 0.006787718912100005
CMI: 0.0180552489424896

Number of features: 92

Number of aggregated features: 4

Number of features: 92

Number of aggregated features: 5

Number of features: 92

Number of aggregated features: 6

Number of features: 92

Number of aggregated features: 7

Number of features: 92

Number of aggregated features: 6

Number of features: 92

Number of aggregated features: 9

Number of features: 92

Number of aggregated features: 5

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 2

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 2

Number of features: 92

Number of aggregated features: 2



selected columns: ['cyclostationary_mean_tg_1w_3', 'cyclostationary_mean_rr_1w_1', 'cyclostationary_mean_tg_16w_3', 'cyclostationary_mean_rr_24w_0', 'cyclostationary_mean_rr_24w_1', 'cyc

## Only snow

In [17]:
target_df_train

,date,mean,median,year,week,mean_std
0,2001-01-05,0.278983,0.00,2001,1,-1.146332
1,2001-01-13,0.494910,0.51,2001,2,0.371173
2,2001-01-21,0.496092,0.51,2001,3,0.379474
3,2001-01-29,0.427992,0.43,2001,5,-0.099118
4,2001-02-06,0.400512,0.41,2001,6,-0.292244
...,...,...,...,...,...,...
406,2009-11-27,0.363952,0.37,2009,48,-0.549184
407,2009-12-05,0.400487,0.40,2009,49,-0.292423
408,2009-12-13,0.506771,0.52,2009,50,0.454529
409,2009-12-21,0.387530,0.53,2009,52,-0.383480


In [3]:
### BOTH
#basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
basins = ['Piemonte_Nord','Piemonte_Sud','Adda','Dora','Ticino']
path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
path_features_snow='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/snow_aggreg_ARPA/all_coord/'

for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')

    eps = 0.001
    snow_actual_path = path_features_snow+basin+'_snowDepth_aggreg_allCoord.csv'
        
    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS', 
                                                                             'cyclostationary_mean_HS_1w',
                                                                             'cyclostationary_mean_HS_4w', 
                                                                             'cyclostationary_mean_HS_8w',
                                                                             'cyclostationary_mean_HS_12w', 
                                                                             'cyclostationary_mean_HS_16w',
                                                                             'cyclostationary_mean_HS_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
        
    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    
    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
    res = {
            "delta" : [], 
            "numSelected" : [], 
            "selectedFeatures" : [] 
        }
        
    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
        
    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]
    
    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    
    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
        

####################Piemonte_Nord####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.278983    0.00  2001     1 -1.146332
1    2001-01-13  0.494910    0.51  2001     2  0.371173
2    2001-01-21  0.496092    0.51  2001     3  0.379474
3    2001-01-29  0.427992    0.43  2001     5 -0.099118
4    2001-02-06  0.400512    0.41  2001     6 -0.292244
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.363952    0.37  2009    48 -0.549184
407  2009-12-05  0.400487    0.40  2009    49 -0.292423
408  2009-12-13  0.506771    0.52  2009    50  0.454529
409  2009-12-21  0.387530    0.53  2009    52 -0.383480
410  2009-12-29  0.279894    0.27  2009    53 -1.139931

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 19

Number of aggregated features: 8

Number of features: 19

Number of aggregated features: 9

Number of features: 19

Number of aggregated features: 9

Num

CMI: 0.011356913637001947
CMI: 0.0019027572668207926
CMI: 0.00454805237325584
CMI: 0.007699514695016285
CMI: 0.0030619277641947273
CMI: 0.015550076637784183
CMI: 0.006343738537786275
CMI: 0.012562477777008474
Highest CMI score: 0.028501857220909088
Adding original feature: 2
CMI: 0.004557471001971958
CMI: 0.01758707125885106
CMI: 0.02308383529125957
CMI: 0.0010671452568803141
CMI: 0.012196243606450163
CMI: 0.004665432531411051
CMI: 0.01134518869498008
CMI: 0.0031965860459245987
CMI: 0.005929244487621166
Highest CMI score: 0.02308383529125957
Adding original feature: 10
CMI: 0.007671947257277323
CMI: 0.0032716760946586154
CMI: 0.0011927143944516216
CMI: 0.006255668852313323
Highest CMI score: 0.007671947257277323
Adding original feature: 1
CMI: 0.001578439743518234
CMI: 0.001165761724650216
Highest CMI score: 0.001578439743518234
Adding original feature: 25
CMI: 0.0004715167352646876
CMI: 0.007536217513409493
CMI: 0.001954224168207841
Highest CMI score: 0.007536217513409493
Adding origi

----- MI Scores -----
[(0, 0.2952953612194227), (4, 0.2800096649013969), (1, 0.2693167342841738), (3, 0.2603664794085629), (6, 0.25701446205886175), (7, 0.23860945539357833), (10, 0.19815668502907816), (9, 0.18800812404496547), (2, 0.17546552232719306), (8, 0.17113211944503068), (5, 0.1680202883412306), (13, 0.1438600954804504), (12, 0.12664648185843275), (11, 0.12456958028513011), (16, 0.11228971442717278), (15, 0.0932113048755538), (14, 0.08664117588742813), (19, 0.06539741683854618), (18, 0.06531736410747108), (17, 0.04915301449338825), (20, 0.043050184000153854)]
Best MI score: 0.2952953612194227
Adding first best original feature: 0
CMI: 0.024710538457889453
CMI: 0.02686732677925613
CMI: 0.016773178140698897
CMI: 0.024457150389453453
Highest CMI score: 0.02686732677925613
Adding original feature: 4
CMI: 0.01050465643320636
Highest CMI score: 0.01050465643320636
Adding original feature: 6
Highest CMI score: -0.0036882955846470056

[0, 4, 6]


Full model and selected features with C

## Temp + Prec + Snow

In [7]:
### Without 12 months
#basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
basins = ['Piemonte_Nord','Piemonte_Sud','Adda','Dora','Ticino']
path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
path_features_snow='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/snow_aggreg_ARPA/all_coord/'
path_features='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/temporal_aggreg/'

for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')

    eps = 0.001
    actual_path = path_features+basin+'_aggreg.csv'
    snow_actual_path = path_features_snow+basin+'_snowDepth_aggreg_allCoord.csv'

    path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
    
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
    
    output,aggregate_trainVal_temp_prec,aggregate_test_temp_prec = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg', 
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w', 
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w', 
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr', 
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w', 
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w', 
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
    
    output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS', 
                                                                             'cyclostationary_mean_HS_1w',
                                                                             'cyclostationary_mean_HS_4w', 
                                                                             'cyclostationary_mean_HS_8w',
                                                                             'cyclostationary_mean_HS_12w', 
                                                                             'cyclostationary_mean_HS_16w',
                                                                             'cyclostationary_mean_HS_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
        
    
    aggregate_trainVal = pd.concat((aggregate_trainVal_snow,aggregate_trainVal_temp_prec),axis=1)
    aggregate_test = pd.concat((aggregate_test_snow,aggregate_test_temp_prec),axis=1)
    
    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    
    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
    train_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_wrapper_best5_train_withSnow.csv'
    val_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_wrapper_best5_val_withSnow.csv'
    test_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_wrapper_best5_test_withSnow.csv'
        
    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)
    
    res = {
            "delta" : [], 
            "numSelected" : [], 
            "selectedFeatures" : [] 
        }
        
    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
        
    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]
    
    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    
    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
    train_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_nonLinCFA_best5_CMI_train_withSnow.csv'
    val_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_nonLinCFA_best5_CMI_val_withSnow.csv'
    test_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_nonLinCFA_best5_CMI_test_withSnow.csv'
            
    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    train_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_nonLinCFA_CMI_train_withSnow.csv'
    val_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_nonLinCFA_CMI_val_withSnow.csv'
    test_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_nonLinCFA_CMI_test_withSnow.csv'
        
    aggregate_trainVal.loc[:410,selected_colnames].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames].to_csv(test_string, index=False)
        

####################Piemonte_Nord####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.278983    0.00  2001     1 -1.146332
1    2001-01-13  0.494910    0.51  2001     2  0.371173
2    2001-01-21  0.496092    0.51  2001     3  0.379474
3    2001-01-29  0.427992    0.43  2001     5 -0.099118
4    2001-02-06  0.400512    0.41  2001     6 -0.292244
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.363952    0.37  2009    48 -0.549184
407  2009-12-05  0.400487    0.40  2009    49 -0.292423
408  2009-12-13  0.506771    0.52  2009    50  0.454529
409  2009-12-21  0.387530    0.53  2009    52 -0.383480
410  2009-12-29  0.279894    0.27  2009    53 -1.139931

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.278983    0.00  2001     1 -1.146332
1    2001-01-13  0.494910    0.51  2001     2  0.

CMI: 0.0359277462700103
CMI: 0.02382045724785256
Highest CMI score: 0.07206180013602545
Adding original feature: 64
CMI: 0.004097826735380533
CMI: 0.01382469702219985
CMI: 0.022804920088717567
CMI: 0.006471799938081668
CMI: 0.017340425033274764
CMI: 0.005242557232930073
CMI: 0.003501818182803096
CMI: 0.006798984089691651
CMI: 0.004047481105508982
CMI: 0.02269412516795624
CMI: 0.019974154745878298
CMI: 0.0013056142082672195
CMI: 0.0012048020924234348
CMI: 0.011879228399980096
CMI: 0.01618542473647555
CMI: 0.004005471289763168
CMI: 0.010352518470938599
CMI: 0.007789886664297113
CMI: 0.002219751359468475
CMI: 0.027737780307943105
CMI: 0.005804630761545193
CMI: 4.868514016087566e-05
CMI: 0.011162802289949675
CMI: 0.01037660794688236
CMI: 0.011205941976104333
CMI: 0.007962921859826944
CMI: 0.011000460206570528
CMI: 0.00042396560488558444
CMI: 0.018661556220897435
CMI: 0.004118284074363182
CMI: 0.0011686348740115593
CMI: 0.004816972950345277
CMI: 0.019282992570370278
CMI: 4.9136954729633864e

CMI: 0.011356913637001947
CMI: 0.0019027572668207926
CMI: 0.00454805237325584
CMI: 0.007699514695016285
CMI: 0.0030619277641947273
CMI: 0.015550076637784183
CMI: 0.006343738537786275
CMI: 0.012562477777008474
CMI: 0.0014976696878495066
Highest CMI score: 0.028501857220909088
Adding original feature: 2
CMI: 0.004557471001971958
CMI: 0.01758707125885106
CMI: 0.02308383529125957
CMI: 0.0010671452568803141
CMI: 0.012196243606450163
CMI: 0.004665432531411051
CMI: 0.01134518869498008
CMI: 0.0031965860459245987
CMI: 0.005929244487621166
Highest CMI score: 0.02308383529125957
Adding original feature: 10
CMI: 0.007671947257277323
CMI: 0.0032716760946586154
CMI: 0.0011927143944516216
CMI: 0.006255668852313323
Highest CMI score: 0.007671947257277323
Adding original feature: 1
CMI: 0.001578439743518234
CMI: 0.001165761724650216
Highest CMI score: 0.001578439743518234
Adding original feature: 25
CMI: 0.0004715167352646876
CMI: 0.007536217513409493
CMI: 0.001954224168207841
Highest CMI score: 0.0075

CMI: 0.03254723152451919
CMI: 0.03224173917336037
CMI: 0.02321809892143742
CMI: 0.03580837673917306
CMI: 0.05144459620410957
CMI: 0.03573094159557916
CMI: 0.03248007530290292
CMI: 0.020195551103674658
CMI: 0.010482422110331763
CMI: 0.005518327098399223
CMI: 0.004149806638309217
CMI: 0.006720953626461662
CMI: 0.0036400905246349313
CMI: 0.0010215428753105327
CMI: 0.006418362640264741
CMI: 0.025026297532601632
CMI: 0.0225837927734672
CMI: 0.05300599515140947
CMI: 0.0011662821891593872
CMI: 0.014154568539090778
CMI: 0.0034282480889311073
CMI: 0.00556266799650787
CMI: 0.0017745605979289925
CMI: 0.0242797556791938
CMI: 0.014184639541659577
CMI: 0.009190369240083013
CMI: 0.006624388253266861
CMI: 0.0030981634182589424
CMI: 0.006282864570508945
CMI: 0.0029264662653806783
CMI: 0.014122997852305863
CMI: 0.006113992740532287
CMI: 0.0030719299225818453
CMI: 0.0024262826772420976
CMI: 0.0010753876543792146
CMI: 0.02810098729551408
CMI: 0.024144132254435985
CMI: 0.012342105593410813
CMI: 0.006700675

Highest CMI score: 0.02686732677925613
Adding original feature: 4
CMI: 0.01050465643320636
CMI: 0.003958875484086188
CMI: 0.0072140253759021
Highest CMI score: 0.01050465643320636
Adding original feature: 6
CMI: 0.0019662718605416574
CMI: 0.0015827816359185731
Highest CMI score: 0.0019662718605416574
Adding original feature: 41
CMI: 0.00013856595464561972
Highest CMI score: 0.00013856595464561972
Adding original feature: 3
Highest CMI score: -0.001135299287123026

[0, 4, 6, 41, 3]


Full model and selected features with CMI

Full aggregate regression train score: 0.2940530674265355, test score: -1.2805750816952206
Aggregate regression train score with FS: 0.07041643049702362, test score: -0.6891718117396826

Full model and best 5 selected features with CMI

Full aggregate regression train score: 0.2940530674265355, test score: -1.2805750816952206
Aggregate regression train score with FS: 0.07041643049702362, test score: -0.6891718117396826
####################Ticino####################

Highest CMI score: 0.021914181655212417
Adding original feature: 57
CMI: 0.010908361371905123
CMI: 0.007542907661369391
CMI: 0.005061658330239249
CMI: 0.0011935791613940372
CMI: 0.0016799219463409232
CMI: 0.002670821427885217
CMI: 0.007000442607198398
CMI: 0.003108722376250761
CMI: 0.00414769034600021
CMI: 0.001562008896103445
Highest CMI score: 0.010908361371905123
Adding original feature: 2
Highest CMI score: -0.004894442474831201

[14, 0, 57, 2]


Full model and selected features with CMI

Full aggregate regression train score: 0.3637813776798461, test score: -0.3041716642835559
Aggregate regression train score with FS: 0.20094525347181302, test score: 0.04903072533524233

Full model and best 5 selected features with CMI

Full aggregate regression train score: 0.3637813776798461, test score: -0.3041716642835559
Aggregate regression train score with FS: 0.20094525347181302, test score: 0.04903072533524233


In [ ]:
### Solo nevi: tutti e tre negativi sia per Piemonte Nord che Sud
### Piemonte Nord: 0.086 per tutti e 3
### Piemonte Sud: 0.09, 0.18, 0.16

# MTL with snow

In [37]:
### targets
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
path_targets = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
targets_df_train = pd.DataFrame()
targets_df_val = pd.DataFrame()
targets_df_test = pd.DataFrame()
targets_df_trainVal = pd.DataFrame()

for basin in basins:
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_targets+basin+'.csv')
    targets_df_train[basin] = target_df_train.mean_std
    targets_df_val[basin] = target_df_val.mean_std
    targets_df_test[basin] = target_df_test.mean_std
    targets_df_trainVal[basin] = target_df_trainVal.mean_std


target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1 -2.546951
1    2001-01-13  0.380618    0.43  2001     2 -0.277191
2    2001-01-21  0.341985    0.38  2001     3 -0.534156
3    2001-01-29  0.322044    0.35  2001     5 -0.666789
4    2001-02-06  0.354954    0.40  2001     6 -0.447894
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.382706    0.40  2009    48 -0.263306
407  2009-12-05  0.409921    0.46  2009    49 -0.082282
408  2009-12-13  0.472087    0.53  2009    50  0.331204
409  2009-12-21  0.324728    0.00  2009    52 -0.648940
410  2009-12-29  0.086512    0.00  2009    53 -2.233412

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.010645    0.00  2001     1 -2.129508
1    2001-01-13  0.206769    0.00  2001     2 -0.927136
2    2001-01-21  0.267313    0.00  2001     3 -

## Dora-Piemonte Nord-Piemonte Sud

In [11]:
### wrapper best 5 features
path_features = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'

best5_wrapper_fulldf_train = pd.DataFrame()
best5_wrapper_fulldf_val = pd.DataFrame()
best5_wrapper_fulldf_test = pd.DataFrame()

for basin in basins:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_wrapper_best5_train_withSnow.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_wrapper_best5_val_withSnow.csv')
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_wrapper_best5_test_withSnow.csv')
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_temp
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_val_temp
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp



In [12]:
### CMI features
path_features = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'

CMI_fulldf_train = pd.DataFrame()
CMI_fulldf_val = pd.DataFrame()
CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test_withSnow.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp
    

In [13]:
### CMI best5 features
path_features = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test_withSnow.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp
    

In [14]:
fulldf_train_withClass = pd.DataFrame()
fulldf_val_withClass = pd.DataFrame()
fulldf_test_withClass = pd.DataFrame()

for i in range(10):
    fulldf_train_withClass = pd.concat((fulldf_train_withClass,pd.concat((CMI_fulldf_train,pd.DataFrame(1+i*np.ones(len(CMI_fulldf_train)),columns=['basin'])),axis=1)),axis=0)
    fulldf_val_withClass = pd.concat((fulldf_val_withClass,pd.concat((CMI_fulldf_val,pd.DataFrame(1+i*np.ones(len(CMI_fulldf_val)),columns=['basin'])),axis=1)),axis=0)
    fulldf_test_withClass = pd.concat((fulldf_test_withClass,pd.concat((CMI_fulldf_test,pd.DataFrame(1+i*np.ones(len(CMI_fulldf_test)),columns=['basin'])),axis=1)),axis=0)


In [18]:
clust_basins = ['Dora', 'Piemonte_Nord', 'Piemonte_Sud']
colnames = [x for x in best5_wrapper_fulldf_train.columns if x.startswith('Dora') or x.startswith('Piemonte_Nord') or x.startswith('Piemonte_Sud')]

best5_wrapper_clusterdf_train_withClass = pd.DataFrame()
best5_wrapper_clusterdf_val_withClass = pd.DataFrame()
best5_wrapper_clusterdf_test_withClass = pd.DataFrame()

for i in range(len(clust_basins)):
    best5_wrapper_clusterdf_train_withClass = pd.concat((best5_wrapper_clusterdf_train_withClass,pd.concat((best5_wrapper_fulldf_train[colnames],pd.DataFrame(1+i*np.ones(len(best5_wrapper_fulldf_train)),columns=['basin'])),axis=1)),axis=0)
    best5_wrapper_clusterdf_val_withClass = pd.concat((best5_wrapper_clusterdf_val_withClass,pd.concat((best5_wrapper_fulldf_val[colnames],pd.DataFrame(1+i*np.ones(len(best5_wrapper_fulldf_val)),columns=['basin'])),axis=1)),axis=0)
    best5_wrapper_clusterdf_test_withClass = pd.concat((best5_wrapper_clusterdf_test_withClass,pd.concat((best5_wrapper_fulldf_test[colnames],pd.DataFrame(1+i*np.ones(len(best5_wrapper_fulldf_test)),columns=['basin'])),axis=1)),axis=0)

for i in range(len(clust_basins)):
    best5_wrapper_clusterdf_train_withClass[clust_basins[i]] = best5_wrapper_clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
    best5_wrapper_clusterdf_val_withClass[clust_basins[i]] = best5_wrapper_clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
    best5_wrapper_clusterdf_test_withClass[clust_basins[i]] = best5_wrapper_clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

best5_wrapper_clusterdf_train_withClass = best5_wrapper_clusterdf_train_withClass.loc[:,best5_wrapper_clusterdf_train_withClass.columns != 'basin']
best5_wrapper_clusterdf_val_withClass = best5_wrapper_clusterdf_val_withClass.loc[:,best5_wrapper_clusterdf_val_withClass.columns != 'basin']
best5_wrapper_clusterdf_test_withClass = best5_wrapper_clusterdf_test_withClass.loc[:,best5_wrapper_clusterdf_test_withClass.columns != 'basin']


In [21]:
targets_df_train_unfolded = pd.DataFrame()
targets_df_val_unfolded = pd.DataFrame()
targets_df_test_unfolded = pd.DataFrame()

for basin in clust_basins:
    targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
    targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
    targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)


In [22]:
model_DPNPS_ohe = LinearRegression()
model_DPNPS_ohe.fit(pd.concat((best5_wrapper_clusterdf_train_withClass,best5_wrapper_clusterdf_val_withClass)),pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)))

res = model_DPNPS_ohe.predict(best5_wrapper_clusterdf_test_withClass.loc[best5_wrapper_clusterdf_test_withClass.Dora==1].values)
print(r2_score(targets_df_test['Dora'].values, res))
res = model_DPNPS_ohe.predict(best5_wrapper_clusterdf_test_withClass.loc[best5_wrapper_clusterdf_test_withClass.Piemonte_Nord==1].values)
print(r2_score(targets_df_test['Piemonte_Nord'].values, res))
res = model_DPNPS_ohe.predict(best5_wrapper_clusterdf_test_withClass.loc[best5_wrapper_clusterdf_test_withClass.Piemonte_Sud==1].values)
print(r2_score(targets_df_test['Piemonte_Sud'].values, res))


-0.2318881490677389
-0.08430042638033441
-0.00022427793868806667


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [23]:
clust_basins = ['Dora', 'Piemonte_Nord', 'Piemonte_Sud']
colnames = [x for x in CMI_fulldf_train.columns if x.startswith('Dora') or x.startswith('Piemonte_Nord') or x.startswith('Piemonte_Sud')]

clusterdf_train_withClass = pd.DataFrame()
clusterdf_val_withClass = pd.DataFrame()
clusterdf_test_withClass = pd.DataFrame()

for i in range(len(clust_basins)):
    clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((CMI_fulldf_train[colnames],pd.DataFrame(1+i*np.ones(len(CMI_fulldf_train)),columns=['basin'])),axis=1)),axis=0)
    clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((CMI_fulldf_val[colnames],pd.DataFrame(1+i*np.ones(len(CMI_fulldf_val)),columns=['basin'])),axis=1)),axis=0)
    clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((CMI_fulldf_test[colnames],pd.DataFrame(1+i*np.ones(len(CMI_fulldf_test)),columns=['basin'])),axis=1)),axis=0)

for i in range(len(clust_basins)):
    clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
    clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
    clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']


In [24]:
targets_df_train_unfolded = pd.DataFrame()
targets_df_val_unfolded = pd.DataFrame()
targets_df_test_unfolded = pd.DataFrame()

for basin in clust_basins:
    targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
    targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
    targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)


In [25]:
model_DPNPS_ohe = LinearRegression()
model_DPNPS_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)),pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)))

res = model_DPNPS_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass.Dora==1].values)
print(r2_score(targets_df_test['Dora'].values, res))
res = model_DPNPS_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass.Piemonte_Nord==1].values)
print(r2_score(targets_df_test['Piemonte_Nord'].values, res))
res = model_DPNPS_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass.Piemonte_Sud==1].values)
print(r2_score(targets_df_test['Piemonte_Sud'].values, res))


-0.4171782877437771
-0.16586872266323027
-0.3353356345507341


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [27]:
clust_basins = ['Dora', 'Piemonte_Nord', 'Piemonte_Sud']
colnames = [x for x in best5_CMI_fulldf_train.columns if x.startswith('Dora') or x.startswith('Piemonte_Nord') or x.startswith('Piemonte_Sud')]

clusterdf_train_withClass = pd.DataFrame()
clusterdf_val_withClass = pd.DataFrame()
clusterdf_test_withClass = pd.DataFrame()

for i in range(len(clust_basins)):
    clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((best5_CMI_fulldf_train[colnames],pd.DataFrame(1+i*np.ones(len(best5_CMI_fulldf_train)),columns=['basin'])),axis=1)),axis=0)
    clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((best5_CMI_fulldf_val[colnames],pd.DataFrame(1+i*np.ones(len(best5_CMI_fulldf_val)),columns=['basin'])),axis=1)),axis=0)
    clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((best5_CMI_fulldf_test[colnames],pd.DataFrame(1+i*np.ones(len(best5_CMI_fulldf_test)),columns=['basin'])),axis=1)),axis=0)

for i in range(len(clust_basins)):
    clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
    clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
    clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']


In [28]:
targets_df_train_unfolded = pd.DataFrame()
targets_df_val_unfolded = pd.DataFrame()
targets_df_test_unfolded = pd.DataFrame()

for basin in clust_basins:
    targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
    targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
    targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)


In [29]:
model_DPNPS_ohe = LinearRegression()
model_DPNPS_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)),pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)))

res = model_DPNPS_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass.Dora==1].values)
print(r2_score(targets_df_test['Dora'].values, res))
res = model_DPNPS_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass.Piemonte_Nord==1].values)
print(r2_score(targets_df_test['Piemonte_Nord'].values, res))
res = model_DPNPS_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass.Piemonte_Sud==1].values)
print(r2_score(targets_df_test['Piemonte_Sud'].values, res))


-0.6080459173843054
-0.23544322028458642
-0.3642241420355754


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


## Adda - Ticino - LambroOlona - OglioIseo

In [30]:
### wrapper best 5 features
path_features = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'

best5_wrapper_fulldf_train = pd.DataFrame()
best5_wrapper_fulldf_val = pd.DataFrame()
best5_wrapper_fulldf_test = pd.DataFrame()

for basin in ['Lambro_Olona','Oglio_Iseo']:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_train.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_val.csv')
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_test.csv')
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_temp
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_val_temp
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp



In [32]:
### wrapper best 5 features

for basin in ['Adda','Ticino']:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_wrapper_best5_train_withSnow.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_wrapper_best5_val_withSnow.csv')
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_wrapper_best5_test_withSnow.csv')
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_temp
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_val_temp
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp



In [34]:
clust_basins = ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino']
colnames = [x for x in best5_wrapper_fulldf_train.columns if x.startswith('Adda') or x.startswith('Lambro_Olona') or x.startswith('Oglio_Iseo') or x.startswith('Ticino')]

best5_wrapper_clusterdf_train_withClass = pd.DataFrame()
best5_wrapper_clusterdf_val_withClass = pd.DataFrame()
best5_wrapper_clusterdf_test_withClass = pd.DataFrame()

for i in range(len(clust_basins)):
    best5_wrapper_clusterdf_train_withClass = pd.concat((best5_wrapper_clusterdf_train_withClass,pd.concat((best5_wrapper_fulldf_train[colnames],pd.DataFrame(1+i*np.ones(len(best5_wrapper_fulldf_train)),columns=['basin'])),axis=1)),axis=0)
    best5_wrapper_clusterdf_val_withClass = pd.concat((best5_wrapper_clusterdf_val_withClass,pd.concat((best5_wrapper_fulldf_val[colnames],pd.DataFrame(1+i*np.ones(len(best5_wrapper_fulldf_val)),columns=['basin'])),axis=1)),axis=0)
    best5_wrapper_clusterdf_test_withClass = pd.concat((best5_wrapper_clusterdf_test_withClass,pd.concat((best5_wrapper_fulldf_test[colnames],pd.DataFrame(1+i*np.ones(len(best5_wrapper_fulldf_test)),columns=['basin'])),axis=1)),axis=0)

for i in range(len(clust_basins)):
    best5_wrapper_clusterdf_train_withClass[clust_basins[i]] = best5_wrapper_clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
    best5_wrapper_clusterdf_val_withClass[clust_basins[i]] = best5_wrapper_clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
    best5_wrapper_clusterdf_test_withClass[clust_basins[i]] = best5_wrapper_clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

best5_wrapper_clusterdf_train_withClass = best5_wrapper_clusterdf_train_withClass.loc[:,best5_wrapper_clusterdf_train_withClass.columns != 'basin']
best5_wrapper_clusterdf_val_withClass = best5_wrapper_clusterdf_val_withClass.loc[:,best5_wrapper_clusterdf_val_withClass.columns != 'basin']
best5_wrapper_clusterdf_test_withClass = best5_wrapper_clusterdf_test_withClass.loc[:,best5_wrapper_clusterdf_test_withClass.columns != 'basin']


In [38]:
targets_df_train_unfolded = pd.DataFrame()
targets_df_val_unfolded = pd.DataFrame()
targets_df_test_unfolded = pd.DataFrame()

for basin in clust_basins:
    targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
    targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
    targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)


In [39]:
model_ALOT_ohe = LinearRegression()
model_ALOT_ohe.fit(pd.concat((best5_wrapper_clusterdf_train_withClass,best5_wrapper_clusterdf_val_withClass)),pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)))

res = model_ALOT_ohe.predict(best5_wrapper_clusterdf_test_withClass.loc[best5_wrapper_clusterdf_test_withClass.Adda==1].values)
print(r2_score(targets_df_test['Adda'].values, res))
res = model_ALOT_ohe.predict(best5_wrapper_clusterdf_test_withClass.loc[best5_wrapper_clusterdf_test_withClass.Lambro_Olona==1].values)
print(r2_score(targets_df_test['Lambro_Olona'].values, res))
res = model_ALOT_ohe.predict(best5_wrapper_clusterdf_test_withClass.loc[best5_wrapper_clusterdf_test_withClass.Oglio_Iseo==1].values)
print(r2_score(targets_df_test['Oglio_Iseo'].values, res))
res = model_ALOT_ohe.predict(best5_wrapper_clusterdf_test_withClass.loc[best5_wrapper_clusterdf_test_withClass.Ticino==1].values)
print(r2_score(targets_df_test['Ticino'].values, res))


-0.065274154848455
-0.07392322281319319
0.005305886280704719
0.0181809008659527


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [41]:
### CMI features
path_features = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'

CMI_fulldf_train = pd.DataFrame()
CMI_fulldf_val = pd.DataFrame()
CMI_fulldf_test = pd.DataFrame()

for basin in ['Lambro_Olona','Oglio_Iseo']:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp
    

In [42]:
### CMI features
path_features = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'

for basin in ['Adda','Ticino']:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test_withSnow.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp
    

In [44]:
clust_basins = ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino']
colnames = [x for x in CMI_fulldf_train.columns if x.startswith('Adda') or x.startswith('Lambro_Olona') or x.startswith('Oglio_Iseo') or x.startswith('Ticino')]

clusterdf_train_withClass = pd.DataFrame()
clusterdf_val_withClass = pd.DataFrame()
clusterdf_test_withClass = pd.DataFrame()

for i in range(len(clust_basins)):
    clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((CMI_fulldf_train[colnames],pd.DataFrame(1+i*np.ones(len(CMI_fulldf_train)),columns=['basin'])),axis=1)),axis=0)
    clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((CMI_fulldf_val[colnames],pd.DataFrame(1+i*np.ones(len(CMI_fulldf_val)),columns=['basin'])),axis=1)),axis=0)
    clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((CMI_fulldf_test[colnames],pd.DataFrame(1+i*np.ones(len(CMI_fulldf_test)),columns=['basin'])),axis=1)),axis=0)

for i in range(len(clust_basins)):
    clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
    clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
    clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']


In [45]:
targets_df_train_unfolded = pd.DataFrame()
targets_df_val_unfolded = pd.DataFrame()
targets_df_test_unfolded = pd.DataFrame()

for basin in clust_basins:
    targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
    targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
    targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)


In [46]:
model_ALOT_ohe = LinearRegression()
model_ALOT_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)),pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)))

res = model_ALOT_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass.Adda==1].values)
print(r2_score(targets_df_test['Adda'].values, res))
res = model_ALOT_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass.Lambro_Olona==1].values)
print(r2_score(targets_df_test['Lambro_Olona'].values, res))
res = model_ALOT_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass.Oglio_Iseo==1].values)
print(r2_score(targets_df_test['Oglio_Iseo'].values, res))
res = model_ALOT_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass.Ticino==1].values)
print(r2_score(targets_df_test['Ticino'].values, res))


0.013986823920808522
0.03628562869164598
0.00023305432522968683
0.09976086057892153


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [48]:
### CMI best5 features
path_features = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in ['Lambro_Olona','Oglio_Iseo']:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp
    

In [49]:
### CMI best5 features
path_features = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'

for basin in ['Adda','Ticino']:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test_withSnow.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp
    

In [50]:
clust_basins = ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino']
colnames = [x for x in best5_CMI_fulldf_train.columns if x.startswith('Adda') or x.startswith('Lambro_Olona') or x.startswith('Oglio_Iseo') or x.startswith('Ticino')]

clusterdf_train_withClass = pd.DataFrame()
clusterdf_val_withClass = pd.DataFrame()
clusterdf_test_withClass = pd.DataFrame()

for i in range(len(clust_basins)):
    clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((best5_CMI_fulldf_train[colnames],pd.DataFrame(1+i*np.ones(len(best5_CMI_fulldf_train)),columns=['basin'])),axis=1)),axis=0)
    clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((best5_CMI_fulldf_val[colnames],pd.DataFrame(1+i*np.ones(len(best5_CMI_fulldf_val)),columns=['basin'])),axis=1)),axis=0)
    clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((best5_CMI_fulldf_test[colnames],pd.DataFrame(1+i*np.ones(len(best5_CMI_fulldf_test)),columns=['basin'])),axis=1)),axis=0)

for i in range(len(clust_basins)):
    clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
    clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
    clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']


In [51]:
targets_df_train_unfolded = pd.DataFrame()
targets_df_val_unfolded = pd.DataFrame()
targets_df_test_unfolded = pd.DataFrame()

for basin in clust_basins:
    targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
    targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
    targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)


In [52]:
model_ALOT_ohe = LinearRegression()
model_ALOT_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)),pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)))

res = model_ALOT_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass.Adda==1].values)
print(r2_score(targets_df_test['Adda'].values, res))
res = model_ALOT_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass.Lambro_Olona==1].values)
print(r2_score(targets_df_test['Lambro_Olona'].values, res))
res = model_ALOT_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass.Oglio_Iseo==1].values)
print(r2_score(targets_df_test['Oglio_Iseo'].values, res))
res = model_ALOT_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass.Ticino==1].values)
print(r2_score(targets_df_test['Ticino'].values, res))


0.05067608676059154
0.07380032998319885
0.03200274171462392
0.13688562773577584


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
